In [1]:
import numpy as np
import math
from vector import Vector

In [2]:
'''
Software to test visible space representation presented in
https://journals.sagepub.com/doi/full/10.5772/63484

Dynamic Path planning on Mobile Robot based on Visible Space and Improved GA.
'''

'\nSoftware to test visible space representation presented in\nhttps://journals.sagepub.com/doi/full/10.5772/63484\n\nDynamic Path planning on Mobile Robot based on Visible Space and Improved GA.\n'

In [33]:
map1 = np.zeros(shape=(5,5))
obstacles = np.array([(2,2),(2,3),(2,4),(3,2),(3,3),(3,4),(4,2),(4,3),(4,4)])
free_space = np.array([(1,1),(1,2),(1,3),(1,4),(1,5),(2,1),(2,5),(3,1),\
                      (3,5),(4,1),(4,5),(5,1),(5,2),(5,3),(5,4),(5,5)])
for i,j in obstacles:
    map1[i-1,j-1] = 1
map1

array([[0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0.],
       [0., 1., 1., 1., 0.],
       [0., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0.]])

In [29]:
def obstacle_threshold(start: (int,int), obstacle: (int,int)): # (x,y) returns (low, high)
    obj_vec = Vector(abs(obstacle[0] - start[0]) , abs(obstacle[1] - start[1]))

    tl = obj_vec + Vector(-0.5,0.5)
    tr = obj_vec + Vector(0.5,0.5)
    bl = obj_vec + Vector(-0.5,-0.5)
    br = obj_vec + Vector(0.5,-0.5)


    ls_angle = np.array([i.angle() for i in [tl,tr,bl,br]])

    min_ls_angle = min(ls_angle)
    max_ls_angle = max(ls_angle)
    return (min_ls_angle, max_ls_angle)

def sign(x):
    if x>=0:
        return 1
    else:
        return -1

def obstacle_obstructed(start: (int,int), end: (int,int), obstacle: (int,int)):
    traj_vector = np.array([end[0] - start[0], end[1] - start[1]])
    obj_vector = np.array([obstacle[0] - start[0], obstacle[1] - start[1]])

    obj_2_traj = traj_vector - obj_vector

    traj_vector = Vector(end[0] - start[0], end[1] - start[1])
    obj_vector = Vector(obstacle[0] - start[0], obstacle[1] - start[1])

    dot = traj_vector * obj_vector

    if dot <= 0: #if it is at or in front of obstacle, then is gud
        return False

    sig_x = sign(obj_vector.x)
    sig_y = sign(obj_vector.y)
    traj_vector.x *= sig_x
    traj_vector.y *= sig_y

    obj_vector.x *= sig_x
    obj_vector.y *= sig_y

    angle_traj = traj_vector.angle()
    threshold = obstacle_threshold(start, obstacle)

    return threshold[0] <= angle_traj <= threshold[1]


Software iterates through the free space given and then creates a subset of the freespace that has no obstacles obstructing it. <br> <br>

The reason behind this is to make sure that the robot does not choose any impossible paths and that it is easier to have a convergence rate with a visible space representation. <br> <br>

This does not follow the outer envelope algorithm described in the paper because I found it quite inaccurate. 

In [38]:
def visible_space(start: (int,int), free_space: [(int,int)], obstacles: [(int,int)]):
    return [space for space in free_space \
            if not np.any([obstacle_obstructed(start, space, obstacle) for obstacle in obstacles])]
            

In [39]:
visible_space( (1,1), free_space, obstacles)

[array([1, 1]),
 array([1, 2]),
 array([1, 3]),
 array([1, 4]),
 array([1, 5]),
 array([2, 1]),
 array([3, 1]),
 array([4, 1]),
 array([5, 1])]